In [1]:
import os
from dotenv import load_dotenv 
load_dotenv("../../constants/.env")

api_key = os.environ.get('BINANCE_KEY')
secret_key = os.environ.get('BINANCE_SECRET')

import time
from binance.client import Client
from binance.enums import *
import pandas as pd
from datetime import datetime

# Configurações da API
API_KEY = api_key
API_SECRET = secret_key
client = Client(API_KEY, API_SECRET)

from pymongo import MongoClient

In [2]:
# Configuração do MongoDB
mongo_client = MongoClient("mongodb://localhost:27017/")
db = mongo_client['trader_data']

# Função para converter datas em timestamps
def date_to_timestamp(date_str):
    return int(datetime.strptime(date_str, "%d/%m/%Y").timestamp() * 1000)

# Função para processar e salvar trades diretamente no MongoDB
def process_and_save_trades(symbol, start_time, end_time, timeframe_seconds=2):
    
    collection = db[f'seconds_{symbol}']

    current_time = start_time
    
    while current_time < end_time:
        try:
            # Obter dados da API
            trades = client.get_aggregate_trades(
                symbol=symbol,
                startTime=current_time,
                limit=1000  # Máximo permitido por requisição
            )

            if not trades:
                break  # Se não há mais trades, encerra o loop

            # Converter trades para DataFrame
            data = [
                {"price": float(trade["p"]), "quantity": float(trade["q"]), "timestamp": trade["T"]}
                for trade in trades
            ]
            df = pd.DataFrame(data)
            df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
            df['timeframe'] = df['datetime'].dt.floor(f'{timeframe_seconds}s')

            # Agrupar por intervalo de tempo e criar candles
            candles = df.groupby('timeframe').agg(
                open=('price', 'first'),
                high=('price', 'max'),
                low=('price', 'min'),
                close=('price', 'last'),
                volume=('quantity', 'sum')
            ).reset_index()
            candles['timestamp'] = candles['timeframe'].astype(int) // 10**9 * 1000

            # Converter candles para formato de dicionário compatível com MongoDB
            records = candles.to_dict(orient='records')

            # Salvar no MongoDB
            if records:
                collection.insert_many(records)
                print(f"Salvo {len(records)} candles no MongoDB. Último candle: {records[-1]['timeframe']}")

            # Atualiza o timestamp para continuar do último trade retornado
            current_time = trades[-1]['T'] + 1  # Avança para o próximo timestamp
            # break
            # Pausa para respeitar limites da API
            time.sleep(0.1)
        except Exception as e:
            print(f"Erro ao processar dados: {e}")
            break




In [3]:

pairs = ['DOGEUSDT','ADAUSDT','XRPUSDT','TRXUSDT','APTUSDT','BTCUSDT',
         'ETHUSDT','AVAXUSDT','BNBUSDT','LTCUSDT','LINKUSDT',
         'EOSUSDT','ETCUSDT','DASHUSDT','ALGOUSDT','SANDUSDT',
         'ARUSDT', 'INJUSDT', 'MASKUSDT', 'CELOUSDT', 'BANDUSDT', 'ALPHAUSDT', 'PERPUSDT',
         'ACHUSDT', 'CHRUSDT', 'BELUSDT']


In [ ]:
for symbol in pairs:
    start_date = "01/01/2024"
    end_date = "01/12/2024"
    
    # Converter datas para timestamps
    start_timestamp = date_to_timestamp(start_date)
    end_timestamp = date_to_timestamp(end_date)
    
    # Processar e salvar os trades diretamente no MongoDB
    print(f"Iniciando download e salvamento no MongoDB para {symbol}...")
    process_and_save_trades(symbol, start_timestamp, end_timestamp, timeframe_seconds=2)
    print("Processo concluído.")

Iniciando download e salvamento no MongoDB para DOGEUSDT...
Salvo 515 candles no MongoDB. Último candle: 2024-01-01 03:38:28
Salvo 433 candles no MongoDB. Último candle: 2024-01-01 04:04:04
Salvo 329 candles no MongoDB. Último candle: 2024-01-01 04:21:44
Salvo 369 candles no MongoDB. Último candle: 2024-01-01 04:43:16
Salvo 502 candles no MongoDB. Último candle: 2024-01-01 05:18:56
Salvo 509 candles no MongoDB. Último candle: 2024-01-01 06:01:08
Salvo 650 candles no MongoDB. Último candle: 2024-01-01 06:57:42
Salvo 602 candles no MongoDB. Último candle: 2024-01-01 07:47:38
Salvo 661 candles no MongoDB. Último candle: 2024-01-01 08:44:56
Salvo 660 candles no MongoDB. Último candle: 2024-01-01 09:35:14
Salvo 636 candles no MongoDB. Último candle: 2024-01-01 10:22:38
Salvo 558 candles no MongoDB. Último candle: 2024-01-01 10:58:40
Salvo 565 candles no MongoDB. Último candle: 2024-01-01 11:33:28
Salvo 510 candles no MongoDB. Último candle: 2024-01-01 12:07:50
Salvo 463 candles no MongoDB. 